#### 1°: Importar json como DF

In [6]:
import pandas as pd
import psycopg2

In [7]:
caminho_do_arquivo = r"C:\Users\Cliente\Desktop\Engenharia_de_dados_com_Python\SGBDS_PYTHON\Postgree\V_OCORRENCIA_AMPLA.json"
df = pd.read_json(caminho_do_arquivo, encoding='utf-8')

In [8]:
df.head(3)

,Numero_da_Ocorrencia,Numero_da_Ficha,Operador_Padronizado,Classificacao_da_Ocorrencia,Data_da_Ocorrencia,Hora_da_Ocorrencia,Municipio,UF,Regiao,Descricao_do_Tipo,...,Lesoes_Desconhecidas_Tripulantes,Lesoes_Desconhecidas_Passageiros,Lesoes_Desconhecidas_Terceiros,Modelo,CLS,Tipo_ICAO,PMD,Numero_de_Assentos,Nome_do_Fabricante,PSSO
0,25626,202096550,MINIST.JUST.SECRET.NAC.DE SEG.PUB.,Acidente,2020-10-08,16:00,CORUMBÁ,MS,Centro-Oeste,FALHA OU MAU FUNCIONAMENTO DO MOTOR,...,NaN,NaN,NaN,AS 350 B2,H1T,AS50,2250.0,6.0,EUROCOPTER FRANCE,verdadeiro
1,26304,202056314,AERO AGRICOLA ROSARIENSE LTDA,Acidente,2020-12-09,21:30:00,SANT'ANA DO LIVRAMENTO,RS,Sul,COMBUSTÍVEL,...,NaN,NaN,NaN,A188B,L1P,C188,1905.0,1.0,CESSNA AIRCRAFT,verdadeiro
2,25763,202041291,KAIO GENARIO FERREIRA DE MELO,Acidente,2020-10-15,11:00,RIO CLARO,RJ,Sudeste,PERDA DE CONTROLE EM VOO,...,NaN,NaN,NaN,R44,H1P,R44,1088.0,4.0,ROBINSON HELICOPTER,verdadeiro


#### 2°: Tratar os dados. Levar em considaração apenas as colunas: ["Numero da ocorrencia", "Classificacao da ocorrencia", "Data da ocorrencia", "Municipio", "UF", "Regiao"]

In [9]:
colunas = ["Numero_da_Ocorrencia", "Classificacao_da_Ocorrencia", "Data_da_Ocorrencia", "Municipio", "UF", "Regiao", "Nome_do_Fabricante", "Modelo"]
df = df[colunas] # Aqui eu tô pegando o meu dataframe e deixando somente com as colunas da variável colunas
# df.rename(columns={'Classificacao_da_Ocorrência': 'Classificacao_da_Ocorrencia'}, inplace=True)

#### 3°: Tirando o acento dos nomes das colunas

#### 4°: Criando banco de dados e tabela "**Foi feito pela interface do postgre**"

#### 5°: Configurando a conexão com o Postgre
#### 6°: Enviar dados para o Postgre

#### 7°: Criar um DELETE da tabela para não ver histórico, mantendo em banco sempre os dados mais recentes

In [10]:
dbname   = 'python'
user     = 'postgres'
password = 'senha123'
host     = 'localhost'
port     = '5432'

# 5°: Conexão com o postgre
conexao = psycopg2.connect(dbname=dbname,
                        user=user,
                        password=password,
                        host=host,
                        port=port)

cursor = conexao.cursor()

# 7°: Deletando a base de dados
cursor.execute("DELETE FROM anac_mapeamento")

# 6°: Enviar dados para o Postgre
# Carga de dados
for indice, coluna_df in df.iterrows():
    cursor.execute("""
                    INSERT INTO anac_mapeamento (
                    ID,
                    Classificacao_da_Ocorrencia,
                    Dt_Ocorrencia,
                    Municipio,
                    UF,
                    Regiao,
                    Fabricante,
                    Modelo
                     ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s)
                """, (
                        coluna_df["Numero_da_Ocorrencia"],
                        coluna_df["Classificacao_da_Ocorrencia"],
                        coluna_df["Data_da_Ocorrencia"],
                        coluna_df["Municipio"],
                        coluna_df["UF"],
                        coluna_df["Regiao"],
                        coluna_df["Nome_do_Fabricante"],
                        coluna_df["Modelo"]
                     )
                  )
conexao.commit()
cursor.close()
conexao.close()